<a href="https://colab.research.google.com/github/jiztastamablastamarang/TwitterSturm/blob/main/TwitterSturm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###CTRL + F9 Щоб запустити весь код, або Runtime => Run all###


##Credentials##
Реєструємо API Twitter та створюємо додаток з можливістю читати та писати повідомлення. Після цього буде згенеровано подібні ключі доступу

In [1]:
KEY = ""
SECRET = ""
TOKEN = ""
TOKEN_SECRET = ""

##Docs##
Гуглодоки зі списком користувачей, фото, контентом

In [2]:
USERS = "https://docs.google.com/spreadsheets/d/1uUF6WGtFl-WBPe_oLdk4gdgFp09aPOAekBWmHdIUnC0/edit#gid=0"
IMGS = "https://drive.google.com/drive/folders/1V7YZjkovjvvJAoTBEpAGp4VwlRWdneXA" # директорія з фото
# IMGS = "https://docs.google.com/spreadsheets/d/1TmQXycbTh7EsgArWT2mqUfQRPXBmidxZB5lsmyFfnRQ/edit#gid=0" #або URL з фото
CONTENT = "https://docs.google.com/spreadsheets/d/1dsdTocyIgQpJYrBLnjxBl9e3Bdw65vAWmpBPorzLlrI/edit#gid=0"

In [3]:
def load_doc(url):
  return pd.read_csv(url.replace("/edit#gid=", "/export?format=csv&gid="))

##Modules##

In [4]:
import tweepy
import pandas as pd
import requests
import os
import glob
import random
import shutil
import time

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [5]:
!pip install -U -q PyDrive

##API##

In [6]:
def swad():
  auth = tweepy.OAuthHandler(KEY, SECRET)
  auth.set_access_token(TOKEN, TOKEN_SECRET)
  api = tweepy.API(auth, 
                   wait_on_rate_limit=True, 
                   wait_on_rate_limit_notify=True)
  try:
      api.verify_credentials()
  except:
      print("ups!")
  return api
  
api = swad()

##Images##

In [7]:
# images = load_doc(IMGS)

# def load_img(url):
#   file_name = "new.jpg"
#   request = requests.get(url, stream=True)
#   if request.status_code==200:
#     with open(file_name, "wb") as file:
#       for chunk in request:
#         file.write(chunk)
#       media = api.media_upload(file_name)
#       os.remove(file_name)
#       return media
#   else:
#     print("ups!")

# imgs = []
# for url in images["URL"]:
#     imgs.append(load_img(url))

Тут може вискочити повідомлення про підтвердження дій с гуглодиском. Треба скопійювати відповідний код и нажати enter

In [8]:
folder_id = IMGS.split("/")[-1]
imgs = []

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

path = "./temp"
try:
    os.makedirs(path)
except: pass

file_list = drive.ListFile(
    {'q': "'{}' in parents".format(folder_id)}).GetList()

for image in file_list:
    fname = os.path.join(path, image["title"])
    new_file = drive.CreateFile({"id": image["id"]})
    new_file.GetContentFile(fname)
    print("uploading: " + image["title"])
    imgs.append(api.media_upload(os.path.join("/content", "temp", image["title"] )))
shutil.rmtree(path)

uploading: photo_2022-02-25_07-17-45.jpg
uploading: photo_2022-02-25_07-17-44.jpg
uploading: 27_02_22_Bucha_fire3.jfif
uploading: 27.02.22_Троєщина.jpeg
uploading: 2022-02-27_Житомир_обстріл.jpg
uploading: Бородянка_Після обстрілів.jpg
uploading: gettyimages-1238718491.jpg
uploading: 275186063_3244844625791550_7558080127054411714_n.jpg
uploading: mariupol.jpeg
uploading: 274660241_3059356657712999_6548491878968273213_n.jpg
uploading: mariupol-2.jpeg
uploading: pid-novoju-kahovkoju-okupanti-rozstrilyali-mashinu-miscevih-zhiteliv.jpeg
uploading: original.jpg
uploading: gettyimages-1238853230.jpg
uploading: 274483441_3271747659721513_1428952723540980161_n.jpg
uploading: Бородянка після обстрілів.jpg
uploading: Харків2.jpg
uploading: Kharkivsky Avenue, Kyiv Andriy Dubchak (Donbas Frontliner).jpg
uploading: 228485_Harkiv_bombarduvanna_960x380_0.jpg
uploading: 19_main.jpg
uploading: Cz-8c_NPtMRVO2gwxO_1TVWBl7Osi-SA.jpg
uploading: FMg41ReXwAQgTa7.jpg
uploading: 24ukraine-briefing-morning-kyiv

##Content##

In [9]:
content = load_doc(CONTENT)
content

,Messedge,Lengh
0,74% of Americans believe that NATO should #Clo...,250
1,"Since the beginning of the war, hundreds of Ru...",277
2,We don’t need the “green light” for NATO count...,225
3,Save the world like you do in the movies! The ...,100
4,Be that leadership that you used to talk about...,119
5,The nuclear crisis is happening in front of yo...,130
6,Freedom is Ukrainian religion. What is yours? ...,129
7,"Russian troops are bombing cities every day, k...",265
8,"Since the beginning of the war, hundreds of Ru...",280
9,"38 kids were killed, 71 wounded, 1.5 mln in th...",275


In [10]:
# Довжина твіта не повинна перевищувати 280 знаків
messeges = [m for m in content["Messedge"].to_list() if len(m)<=280]

##Test tweet##

In [11]:
def add_comment(user, status, media):
    #останній твіт цільового користувача, який будемо коментувати
    last_tweet = api.user_timeline(screen_name=user, count=1, include_rts=False, tweet_mode='extended')[0]

    #створюємо повідомлення та додаємо фото
    api.update_status(
                      status=status, 
                      in_reply_to_status_id=last_tweet.id, 
                      media_ids=[media]
    )

In [12]:
user = "jiztastamablast" # тестуємо на собі. пишемо свій нік
status= "!!!" #  комент
media = random.choice(imgs).media_id # рандомне фото

In [13]:
api.update_status(status="!", media_ids=[random.choice(imgs).media_id]) 

In [14]:
add_comment(user, status, media)

##Users##

In [15]:
users = load_doc(USERS)
try:
  users["TwitterNickname"] = users["Twitter"].str.replace("https://twitter.com/", "", regex=False).str.replace("@", "", regex=False)
except:
  pass
#прибираємо нульові строки
users.dropna(subset=["TwitterNickname"], inplace=True)
users

,State,Member of Congress,Name,Party,Twitter,Facebook,TwitterNickname
0,Alabama,U.S. Senator,"Shelby, Richard",R,@SenShelby,x,SenShelby
1,Alabama,U.S. Senator,"Tuberville, Tommy",R,@Ttuberville,x,Ttuberville
2,Alabama 1st District,U.S. Representative,"Carl, Jerry",R,@RepJerryCarl,x,RepJerryCarl
3,Alabama 2nd District,U.S. Representative,"Moore, Barry",R,@RepBarryMoore,x,RepBarryMoore
4,Alabama 3rd District,U.S. Representative,"Rogers, Mike",R,@RepMikeRogers,x,RepMikeRogers
...,...,...,...,...,...,...,...
536,Wisconsin 7th District,U.S. Representative,"Tiffany, Thomas",R,@TomTiffanyWI,x,TomTiffanyWI
537,Wisconsin 8th District,U.S. Representative,"Gallagher, Mike",R,@MikeforWI,x,MikeforWI
538,Wyoming,U.S. Senator,"Lummis, Cynthia",R,@SenLummis,x,SenLummis
539,Wyoming,U.S. Senator,"Barrasso, John",R,@SenJohnBarrasso,x,SenJohnBarrasso


In [16]:
# users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 538 entries, 0 to 540
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   State               538 non-null    object
 1   Member of Congress  538 non-null    object
 2   Name                538 non-null    object
 3   Party               538 non-null    object
 4   Twitter             538 non-null    object
 5   Facebook            537 non-null    object
 6   TwitterNickname     538 non-null    object
dtypes: object(7)
memory usage: 33.6+ KB


##Sturm##

In [ ]:
bad_user = []

for user in users["TwitterNickname"].to_list():
  try:
    # обираємо рандомні повідомлення та фото
    status= random.choice(messeges)
    media = random.choice(imgs).media_id

    # додаємо комент
    add_comment(user, status, media)

  except tweepy.TweepError as error:
    if error.api_code == 185:
      print("Перевищено ліміт 300 повідомельнь за 3 години. Чекаємо...")
      time.sleep(3*60*60)
    else:
      print(error)
      print(f"error sending comment to user: {user}")

    bad_user.append(user)
